# Part II

## Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config2 import g_key

### 2. Import the WeatherPy_database.csv file from Part 1 as a new DataFrame.

In [5]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv", index_col=0)
city_data_df.head()


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions,Rain (inches last hour),Snow (inches last hour)
0,Sulam,IL,32.61,35.33,75.99,83,9,1.99,clear sky,0.00,0
1,Namatanai,PG,-3.67,152.43,86.16,65,4,5.82,clear sky,0.00,0
2,Bluff,NZ,-46.60,168.33,48.27,79,95,16.93,light rain,0.23,0
3,Santa Cruz,BO,-17.80,-63.17,59.00,93,90,1.01,overcast clouds,0.00,0
4,San Patricio,MX,19.22,-104.70,80.24,80,100,6.11,light rain,0.19,0


### 3. Filter the DataFrame

In [6]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [28]:
# Ask the customer to answer if he or she would like it to be raining or not 
raining = str(input("Do you want it to be raining? (yes/no) "))
# Ask the customer to answer if he or she would like it to be snowing or not 
snowing = str(input("Do you want it to be snowing? (yes/no) "))

Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [42]:
for index, row in city_data_df.iterrows():
    if raining == "no":
        city_data_df["Rain (inches last hour)"] == 0
    else:
        city_data_df["Rain (inches last hour)"] != 0
    if snowing == "no":
        city_data_df["Snow (inches last hour)"] == 0
    else:
        city_data_df["Snow (inches last hour)"] != 0

In [43]:
rainfall

0       True
1       True
2      False
3       True
4      False
       ...  
565     True
566     True
567     True
568     True
569     True
Name: Rain (inches last hour), Length: 570, dtype: bool

In [27]:
# Filter the dataset to find the cities that fit the criteria.

preferred_cities_df = city_data_df.loc[(city_data_df["Rain (inches last hour)"] != rainfall) & 
                                       (city_data_df["Snow (inches last hour)"] != snowfall) & 
                                       (city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions,Rain (inches last hour),Snow (inches last hour)
0,Sulam,IL,32.61,35.33,75.99,83,9,1.99,clear sky,0.00,0
1,Namatanai,PG,-3.67,152.43,86.16,65,4,5.82,clear sky,0.00,0
4,San Patricio,MX,19.22,-104.70,80.24,80,100,6.11,light rain,0.19,0
9,Kampong Chhnang,KH,12.17,104.55,86.43,66,63,3.58,light rain,0.47,0
14,Pokhara,NP,28.23,83.98,78.12,76,93,3.74,light rain,0.77,0
15,Phuket,TH,8.00,98.25,86.00,74,20,1.12,few clouds,0.00,0
19,Dharur,IN,18.82,76.12,76.82,81,99,10.02,light rain,0.19,0
20,Aswan,EG,24.09,32.91,82.17,20,3,7.72,clear sky,0.00,0
23,Kapaa,US,22.08,-159.32,82.40,69,1,9.17,clear sky,0.00,0
26,Atuona,PF,-9.80,-139.03,80.47,76,3,14.47,clear sky,0.00,0
